In [2]:
import json
import pandas as pd
import time
import requests as r 
import yaml

In [47]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

tenant_id = config['tenant_id']


def get_access_token(tenant_id, client_id, client_secret):
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {
        'client_id': client_id,
        'scope': 'https://graph.microsoft.com/.default',
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = r.post(url, headers=headers, data=data)
    response.raise_for_status()
    return response.json().get('access_token')


list_additional_datasets_id = \
    [
        ['configuredBy_test','dataset_name_test','dataset_id_test']
    ] 

In [ ]:
access_token = get_access_token(tenant_id, config['client_id'], config['client_secret'])

header = {'Authorization': f'Bearer {access_token}'}

# Datasets
responce_groups_datasets = r.get(f'https://api.powerbi.com/v1.0/myorg/groups/{groupId}/datasets', headers=header)
responce_groups_datasets = json.loads(responce_groups_datasets.content)
df = pd.json_normalize(responce_groups_datasets['value'])

df_short = df.sort_values('createdDate', ascending=False)[['configuredBy','name','id']].copy()
df_short_add = pd.DataFrame(data=list_additional_datasets_id, columns=['configuredBy','name','id'])
df_short = pd.concat([df_short, df_short_add], ignore_index=True)
dict_configuredBy = dict(zip(df_short.name, df_short.configuredBy))
print('Datasets done')

# Datasources
url_datasets = 'https://api.powerbi.com/v1.0/myorg/datasets/'

dict_tmp = {}
for name, id in zip(df_short.name.to_list(), df_short.id.to_list()):
    groups = r.get(url_datasets + id + '/datasources', headers=header)
    time.sleep(1)
    groups = json.loads(groups.content)
    dict_tmp[name] = groups['value']

df_result_grouped = pd.DataFrame.from_dict(data = dict_tmp.values())

list_rows = []
for x in dict_tmp:
    for row in dict_tmp[x]:
        list_rows.append([x, dict_configuredBy[x], row['datasourceType'], row['connectionDetails'], row['datasourceId'], row['gatewayId']])

df_datasources = pd.DataFrame(data=list_rows, columns=['Dataset Name', 'configuredBy', 'datasourceType', 'connectionDetails', 'datasourceId', 'gatewayId'])
print('Datasources done')


# Refreshes
dict_ref = {}

for name, id in zip(df_short.name.to_list(), df_short.id.to_list()):
    groups = r.get(url_datasets + id + '/refreshes', headers=header)
    time.sleep(1)
    groups = json.loads(groups.content)
    if 'error' in groups.keys():
        dict_ref[name] = 'Invalid dataset'
    else:
        if len(groups['value']) == 0:
            dict_ref[name] = 'null'
        else:
            dict_ref[name] = groups['value'][0]

list_columns = ['Dataset Name','startTime','status','refreshType', 'errorCode']

list_rows = []
for x, row in zip(df_short.name.to_list(), dict_ref.values()):
    if row == 'null':
        list_rows.append([x, pd.NA, '-', '-','-'])
    elif row == 'Invalid dataset':
        list_rows.append([x, pd.NA, '-', '-','Invalid dataset'])
    elif type(row) == str:
        list_rows.append([x, pd.NA, '-', '-','-'])
    else:
        if 'serviceExceptionJson' in list(row.keys()):
            list_rows.append([x, row['startTime'], row['status'], row['refreshType'], json.loads(row['serviceExceptionJson'])['errorCode']])
        else:
            list_rows.append([x, row['startTime'], row['status'], row['refreshType'],'-'])


df_result_ref = pd.DataFrame(data=list_rows, columns=list_columns)
df_result_ref.startTime = pd.to_datetime(df_result_ref.startTime).dt.strftime('%Y-%m-%d %H:%M:%S')
print('Refreshes done')



# RefreshSchedule
dict_tmp_ref_sched = {}

for name, id in zip(df_short.name.to_list(), df_short.id.to_list()):
    groups = r.get(url_datasets + id + '/refreshSchedule', headers=header)
    time.sleep(1)
    groups = json.loads(groups.content)
    if 'error' in list(groups.keys()):
        dict_tmp_ref_sched[name] = 'error'
    else: 
        dict_tmp_ref_sched[name] = groups['enabled']
print('RefreshSchedule done')


# Gateways
url_getways = 'https://api.powerbi.com/v1.0/myorg/gateways/'
list_gatewayId = df_datasources.gatewayId.drop_duplicates().to_list()

list_columns = ['gatewayId','gateway_name','gatewayMachine']
list_rows = []

for id in list_gatewayId:
    gateway = r.get(url_getways + id, headers=header)
    time.sleep(1)
    if gateway.reason == 'OK':
        gateway = json.loads(gateway.content)
        list_rows.append([id, gateway['name'], json.loads(gateway['gatewayAnnotation'])['gatewayMachine']])
        # print(gateway['name'])
        # print(json.loads(gateway['gatewayAnnotation'])['gatewayMachine'])
    elif gateway.reason == 'Unauthorized':
        list_rows.append([id,'Unauthorized', '-'])
    else:
        list_rows.append([id,'Personal cloud connect', '-'])
    
df_geteways = pd.DataFrame(data=list_rows, columns = list_columns)
print('Gateways done')



# Final Result 
df_result_ref['enabled'] = dict_tmp_ref_sched.values()
df_final_result = pd.merge(df_datasources, df_result_ref, how='left', on='Dataset Name')
df_final_result = pd.merge(df_final_result, df_geteways, how='left', on='gatewayId')
list_columns_order = ['Dataset Name','configuredBy','startTime', 'status', 'enabled','gateway_name'
                        , 'gatewayMachine', 'datasourceType','connectionDetails'
                        ,'refreshType','datasourceId','gatewayId']

df_final_result[list_columns_order].sort_values('startTime',ascending=False).to_excel('result.xlsx',index=False)